In [2]:
import geopandas as gpd

gdb_path = "../data/raw/ICADisplay.gdb"
feeder = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederDetail")
# feeder_load = gpd.read_file(gdb_path, driver='fileGDB', layer="FeederLoadProfile")
line = gpd.read_file(gdb_path, driver='fileGDB', layer="LineDetail")
substations = gpd.read_file(gdb_path, driver='fileGDB', layer="Substations")
# substation_load = gpd.read_file(gdb_path, driver='fileGDB', layer="SubstationLoadProfile")

In [4]:
# Convert to GeoJSON and save to file
feeder = feeder.to_crs(epsg=4326)
feeder.to_file("../data/geojson/feeder.geojson", driver='GeoJSON')

In [10]:
# Feeder less than 100 kW
import pandas as pd

aggregated_capacity = line.groupby('FeederId').agg({'LoadCapacity_kW': 'sum'}).reset_index()
feeder_capacity = pd.merge(feeder, aggregated_capacity, left_on='FeederID', right_on='FeederId', how='inner')

capacity = 100 # 100 kW = low
low_capacity_feeders = feeder_capacity[feeder_capacity['LoadCapacity_kW'] < capacity]

low_capacity_lines = pd.merge(line, low_capacity_feeders[['FeederID']], left_on='FeederId', right_on='FeederID', how='inner')
low_capacity_lines_gdf = gpd.GeoDataFrame(low_capacity_lines, geometry='geometry')

low_capacity_lines_gdf.to_file("../data/geojson/low_capacity_feeders.geojson", driver='GeoJSON')

In [11]:
line = line.to_crs(epsg=4326)
line.to_file("../data/geojson/line.geojson", driver='GeoJSON')

In [12]:
substations = substations.to_crs(epsg=4326)
substations.to_file("../data/geojson/substations.geojson", driver='GeoJSON')
# substation_load.to_file("../data/geojson/substation_load.geojson", driver='GeoJSON')